# Hyper parameter tuning
## Part 2: Testing different model architectures and finding best hyper parameters

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from os import walk
import time
import regex as re
import os
import glob
import tensorflow as tf
import warnings
import matplotlib
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

# Reading Data Files

In [2]:
def read_many_hdf5(num_images, file_path):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Open the HDF5 file
    file = h5py.File(file_path, "r+")

    images = np.array(file["/images"]).astype("uint8")
    labels = np.array(file["/meta"]).astype("uint8")

    return images, labels

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dir_file_path = "/content/drive/MyDrive/Data_HDF/Porosity_Balanced_Data.h5"
X, y = read_many_hdf5(0,dir_file_path)
print(X.shape)
print(len(y))
(unique, counts) = np.unique(y, return_counts=True)
print("Total non-porosity images in balanced dataset: ", counts[0])
print("Total porosity images in balanced dataset: ", counts[1])

(5135, 190, 150, 3)
5135
Total non-porosity images in balanced dataset:  2578
Total porosity images in balanced dataset:  2557


__Otsu Binarisation__

In [5]:
def ostu_thd(XX):
    rnd_images = []
    for img in XX:
        #img = rgb2gray(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Otsu's thresholding
        ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        rnd_images.append(th2)
    test_images = np.array(rnd_images)
    
    return test_images

In [6]:
print("X Shape Before Ostu Binarisation: " + str(X.shape))
X = ostu_thd(X)
print("X Shape after Ostu Binarisation: " + str(X.shape))

X Shape Before Ostu Binarisation: (5135, 190, 150, 3)
X Shape after Ostu Binarisation: (5135, 190, 150)


__Pre Processing__

In [7]:
from sklearn.model_selection import train_test_split

# test_size is the test size of the total dataset, random_state controls the shuffling of data. same state returns same data always.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=786, stratify=y) 
print("Total training intences: " + str(len(y_train)))
print("Total testing intences: " + str(len(y_test)))

print("Before Reshaping the shape of train and test dataset:")
print(X_train.shape)
print(X_test.shape)
# reshape to be [samples][width][height][channels]
X_train = X_train.reshape((X_train.shape[0], 190, 150, 1))
X_test = X_test.reshape((X_test.shape[0], 190, 150, 1))
print("After Reshaping the shape of train and test dataset:")
print(X_train.shape)
print(X_test.shape)
# Convert the array to float32 as opposed to uint8
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# Convert the pixel values from integers between 0 and 255 to floats between 0 and 1
X_train /= 255
X_test /=  255
NUM_DIGITS = 2

print("Before", y_train[0]) # The format of the labels before conversion

y_train  = tf.keras.utils.to_categorical(y_train, NUM_DIGITS)

print("After", y_train[0]) # The format of the labels after conversion

y_test = tf.keras.utils.to_categorical(y_test, NUM_DIGITS)

Total training intences: 3594
Total testing intences: 1541
Before Reshaping the shape of train and test dataset:
(3594, 190, 150)
(1541, 190, 150)
After Reshaping the shape of train and test dataset:
(3594, 190, 150, 1)
(1541, 190, 150, 1)
Before 1
After [0. 1.]


In [8]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 41.9 MB/s 


# Keras Tuner

In [29]:
import keras_tuner as kt
from keras_tuner import RandomSearch, Hyperband, BayesianOptimization
from tensorflow import keras
from tensorflow.keras import layers

## Model 1
__Define Model Parameters__

Model Architeture<br>

Convolutiona layers = 1 <br>
Maxpooling layers   = 1<br>
Dense layers        = 1<br>

In [30]:
def build_model_1c1p1f1d1d_basic(hp):
    model = keras.Sequential()
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_1_kernel', values=[3,5,7]),
        activation = hp.Choice(name = 'activation_1_conv', values = ['relu', 'elu', 'tanh' ], ordered = False), #'selu'
        #activation = 'relu',
        input_shape = (190, 150,1))
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_1_stride', min_value=2, max_value=12, step=2),
        )
    )
    
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("dense_1_units", min_value=32, max_value=264, step=64),
            #activation = 'relu',
            activation = hp.Choice(name = 'activation_1_dense', values = ['relu','tanh' ], ordered = False),
        )
    )
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-4, 1e-3, 1e-2])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

__Hyperband Tuner__

In [31]:
SEED = 768
tuner = Hyperband(
    build_model_1c1p1f1d1d_basic,
    objective="val_accuracy",
    max_epochs=10,
    hyperband_iterations = 1,
    seed=SEED,
    overwrite=True,
    directory="my_dir",
    project_name="HyperbandTuner_basic",
)

__Search Space__

In [32]:
tuner.search_space_summary()

Search space summary
Default search space size: 7
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 96, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5, 7], 'ordered': True}
activation_1_conv (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'elu', 'tanh'], 'ordered': False}
MaxPol_1_stride (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 12, 'step': 2, 'sampling': None}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 264, 'step': 64, 'sampling': None}
activation_1_dense (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 0.001, 0.01], 'ordered': True}


__Search the best parameters__

In [33]:
tuner.search(X_train, y_train, validation_split=0.3, batch_size=20)

Trial 30 Complete [00h 00m 21s]
val_accuracy: 0.9675625562667847

Best val_accuracy So Far: 0.9823911190032959
Total elapsed time: 00h 09m 19s


__Results of experiments__

In [34]:
tuner.results_summary(num_trials=30)

Results summary
Results in my_dir/HyperbandTuner_basic
Showing 30 best trials
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 7
activation_1_conv: relu
MaxPol_1_stride: 2
dense_1_units: 160
activation_1_dense: relu
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9823911190032959
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 5
activation_1_conv: relu
MaxPol_1_stride: 2
dense_1_units: 160
activation_1_dense: relu
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 0.9740500450134277
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 7
activation_1_conv: relu
MaxPol_1_stride: 8
dense_1_units: 224
activation_1_dense: relu
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0014
Score: 0.9684893488883972
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 3

In [35]:
rm -rf my_dir/

## Model 2
__Define Model Parameters__

Model Architeture<br>

Convolutiona layers = 1<br>
Maxpooling layers   = 1<br>
Dense layers        = 2<br>

In [36]:
def build_model_1c1p1f2d1d_basic(hp):
    model = keras.Sequential()
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_1_kernel', values=[3,5,7]),
        activation = hp.Choice(name = 'activation_1_conv', values = ['relu', 'elu', 'tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        input_shape = (190, 150,1))
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_1_stride', min_value=2, max_value=12, step=2),
        )
    )
    
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("dense_1_units", min_value=32, max_value=264, step=64),
            #activation = 'relu',
            activation = hp.Choice(name = 'activation_1_dense', values = ['relu','tanh' ], ordered = False, default='relu'),
        )
    )
    model.add(
        layers.Dense(
            units=hp.Int("dense_2_units", min_value=32, max_value=264, step=64),
            #activation = 'relu',
            activation = hp.Choice(name = 'activation_2_dense', values = ['relu','tanh' ], ordered = False, default='relu'),
        )
    )
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-4, 1e-3, 1e-2])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [37]:
SEED = 768
tuner = Hyperband(
    build_model_1c1p1f2d1d_basic,
    objective="val_accuracy",
    max_epochs=10,
    hyperband_iterations = 1,
    seed=SEED,
    overwrite=True,
    directory="my_dir",
    project_name="HyperbandTuner_basic",
)

tuner.search_space_summary()

tuner.search(X_train, y_train, validation_split=0.3, batch_size=20)

Trial 30 Complete [00h 00m 28s]
val_accuracy: 0.9759036302566528

Best val_accuracy So Far: 0.9777571558952332
Total elapsed time: 00h 08m 30s


In [38]:
tuner.results_summary(num_trials=100)
!rm -rf my_dir/ 

Results summary
Results in my_dir/HyperbandTuner_basic
Showing 100 best trials
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 7
activation_1_conv: relu
MaxPol_1_stride: 2
dense_1_units: 160
activation_1_dense: relu
dense_2_units: 224
activation_2_dense: tanh
learning_rate: 0.001
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9777571558952332
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 7
activation_1_conv: relu
MaxPol_1_stride: 8
dense_1_units: 160
activation_1_dense: relu
dense_2_units: 32
activation_2_dense: tanh
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.9768304228782654
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 5
activation_1_conv: elu
MaxPol_1_stride: 6
dense_1_units: 160
activation_1_dense: tanh
dense_2_units: 224
activation_2_dense: tanh
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Sc

## Model 3
__Define Model Parameters__

Model Architeture<br>

Convolutiona layers = 2<br>
Maxpooling layers   = 2<br>
Dense layers        = 1<br>

In [39]:
def build_model_2c2p1f1d1d_basic(hp):
    model = keras.Sequential()
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_1_kernel', values=[3,5,7]),
        activation = hp.Choice(name = 'activation_1_conv', values = ['relu', 'elu','tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        input_shape = (190, 150,1))
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_1_stride', min_value=2, max_value=12, step=2),
        )
    )
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_2_kernel', values=[3,5,7]),
        activation = hp.Choice(name = 'activation_2_conv', values = ['relu', 'elu','tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        )
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_2_stride', min_value=2, max_value=12, step=2),
        )
    )
    
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("dense_1_units", min_value=32, max_value=264, step=64),
            #activation = 'relu',
            activation = hp.Choice(name = 'activation_1_dense', values = [ 'relu', 'tanh'], ordered = False, default='relu'),
        )
    )
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-4, 1e-3, 1e-2])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [40]:
SEED = 768
tuner = Hyperband(
    build_model_2c2p1f1d1d_basic,
    objective="val_accuracy",
    max_epochs=10,
    hyperband_iterations = 1,
    seed=SEED,
    overwrite=True,
    directory="my_dir",
    project_name="HyperbandTuner_basic",
)

tuner.search_space_summary()

tuner.search(X_train, y_train, validation_split=0.3, batch_size=20)

Trial 30 Complete [00h 00m 29s]
val_accuracy: 0.9666357636451721

Best val_accuracy So Far: 0.9805375337600708
Total elapsed time: 00h 07m 20s


In [41]:
tuner.results_summary(num_trials=100)
!rm -rf my_dir/ 

Results summary
Results in my_dir/HyperbandTuner_basic
Showing 100 best trials
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 7
activation_1_conv: relu
MaxPol_1_stride: 6
conv_2_filter: 80
conv_2_kernel: 5
activation_2_conv: tanh
MaxPol_2_stride: 4
dense_1_units: 224
activation_1_dense: tanh
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9805375337600708
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 3
activation_1_conv: tanh
MaxPol_1_stride: 4
conv_2_filter: 96
conv_2_kernel: 3
activation_2_conv: relu
MaxPol_2_stride: 4
dense_1_units: 224
activation_1_dense: tanh
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9731232523918152
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 7
activation_1_conv: relu
MaxPol_1_stride: 2
conv_2_filter: 48
conv_2_kernel: 7
activation_2_conv: tanh
MaxPol_2_stride: 10
dense_1_units: 160
activation_1_

## Model 4
__Define Model Parameters__

Model Architeture<br>

Convolutiona layers = 2<br>
Maxpooling layers   = 2<br>
Dense layers        = 2<br>

In [42]:
def build_model_2c2p1f2d1d_basic(hp):
    model = keras.Sequential()
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_1_kernel', values=[3,5,7]),
        activation = hp.Choice(name = 'activation_1_conv', values = ['relu', 'elu','tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        input_shape = (190, 150,1))
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_1_stride', min_value=2, max_value=12, step=2),
        )
    )
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_2_kernel', values=[3,5,7]),
        activation = hp.Choice(name = 'activation_2_conv', values = ['relu', 'elu','tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        )
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_2_stride', min_value=2, max_value=12, step=2),
        )
    )
    
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("dense_1_units", min_value=32, max_value=264, step=64),
            #activation = 'relu',
            activation = hp.Choice(name = 'activation_1_dense', values = [ 'relu', 'tanh'], ordered = False, default='relu'),
        )
    )
    model.add(
        layers.Dense(
            units=hp.Int("dense_2_units", min_value=32, max_value=264, step=64),
            #activation = 'relu',
            activation = hp.Choice(name = 'activation_2_dense', values = ['relu','tanh' ], ordered = False, default='relu'),
        )
    )
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-4, 1e-3, 1e-2])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [43]:
SEED = 768
tuner = Hyperband(
    build_model_2c2p1f2d1d_basic,
    objective="val_accuracy",
    max_epochs=10,
    hyperband_iterations = 1,
    seed=SEED,
    overwrite=True,
    directory="my_dir",
    project_name="HyperbandTuner_basic",
)

tuner.search_space_summary()

tuner.search(X_train, y_train, validation_split=0.3, batch_size=20)

Trial 30 Complete [00h 00m 24s]
val_accuracy: 0.9481000900268555

Best val_accuracy So Far: 0.9759036302566528
Total elapsed time: 00h 07m 17s


In [44]:
tuner.results_summary(num_trials=100)
!rm -rf my_dir/ 

Results summary
Results in my_dir/HyperbandTuner_basic
Showing 100 best trials
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 3
activation_1_conv: relu
MaxPol_1_stride: 2
conv_2_filter: 48
conv_2_kernel: 7
activation_2_conv: tanh
MaxPol_2_stride: 6
dense_1_units: 96
activation_1_dense: tanh
dense_2_units: 96
activation_2_dense: relu
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0020
Score: 0.9759036302566528
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 5
activation_1_conv: relu
MaxPol_1_stride: 6
conv_2_filter: 48
conv_2_kernel: 5
activation_2_conv: tanh
MaxPol_2_stride: 2
dense_1_units: 160
activation_1_dense: relu
dense_2_units: 96
activation_2_dense: tanh
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0019
Score: 0.9749768376350403
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 3
activation_1_conv: relu
MaxP

## Model 5
__Define Model Parameters__

Model Architeture<br>

Convolutiona layers = 3<br>
Maxpooling layers   = 3<br>
Dense layers        = 1<br>

In [45]:
def build_model_3c3p1f1d1d_basic(hp):
    model = keras.Sequential()
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_1_kernel', values=[3,5,7,9,11]),
        activation = hp.Choice(name = 'activation_1_conv', values = ['relu', 'elu','tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        input_shape = (190, 150,1))
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_1_stride', min_value=2, max_value=8, step=2),
        )
    )
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_2_kernel', values=[3,5]),
        activation = hp.Choice(name = 'activation_2_conv', values = ['relu', 'elu','tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        )
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_2_stride', min_value=2, max_value=4, step=2),
        )
    )
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_3_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_3_kernel', values=[2,3]),
        activation = hp.Choice(name = 'activation_3_conv', values = ['relu', 'elu','tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        )
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_3_stride', min_value=2, max_value=2, step=2),
        )
    )
    
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("dense_1_units", min_value=32, max_value=264, step=64),
            #activation = 'relu',
            activation = hp.Choice(name = 'activation_1_dense', values = ['relu','tanh' ], ordered = False, default='relu'),
        )
    )
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-4, 1e-3, 1e-2])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [46]:
SEED = 768
tuner = Hyperband(
    build_model_3c3p1f1d1d_basic,
    objective="val_accuracy",
    max_epochs=10,
    hyperband_iterations = 1,
    seed=SEED,
    overwrite=True,
    directory="my_dir",
    project_name="HyperbandTuner_basic",
)

tuner.search_space_summary()

tuner.search(X_train, y_train, validation_split=0.3, batch_size=20)

Trial 30 Complete [00h 00m 21s]
val_accuracy: 0.5115848183631897

Best val_accuracy So Far: 0.9796107411384583
Total elapsed time: 00h 09m 09s


In [47]:
tuner.results_summary(num_trials=100)
!rm -rf my_dir/ 

Results summary
Results in my_dir/HyperbandTuner_basic
Showing 100 best trials
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 3
activation_1_conv: relu
MaxPol_1_stride: 2
conv_2_filter: 32
conv_2_kernel: 5
activation_2_conv: tanh
MaxPol_2_stride: 4
conv_3_filter: 80
conv_3_kernel: 3
activation_3_conv: tanh
MaxPol_3_stride: 2
dense_1_units: 224
activation_1_dense: tanh
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0023
Score: 0.9796107411384583
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 7
activation_1_conv: elu
MaxPol_1_stride: 2
conv_2_filter: 80
conv_2_kernel: 5
activation_2_conv: tanh
MaxPol_2_stride: 2
conv_3_filter: 48
conv_3_kernel: 2
activation_3_conv: relu
MaxPol_3_stride: 2
dense_1_units: 224
activation_1_dense: tanh
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9786839485168457
Trial summary
Hyperparameters:
conv_1_filte

## Model 6
__Define Model Parameters__

Model Architeture<br>

Convolutiona layers = 3<br>
Maxpooling layers   = 3<br>
Dense layers        = 2<br>

In [48]:
def build_model_3c3p1f2d1d_basic(hp):
    model = keras.Sequential()
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_1_kernel', values=[3,5,7,9,11]),
        activation = hp.Choice(name = 'activation_1_conv', values = ['relu', 'elu','tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        input_shape = (190, 150,1))
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_1_stride', min_value=2, max_value=8, step=2),
        )
    )
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_2_kernel', values=[3,5]),
        activation = hp.Choice(name = 'activation_2_conv', values = ['relu', 'elu','tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        )
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_2_stride', min_value=2, max_value=4, step=2),
        )
    )
    model.add(
        layers.Conv2D(
        filters = hp.Int('conv_3_filter', min_value=32, max_value=96, step=16),
        kernel_size = hp.Choice('conv_3_kernel', values=[2,3]),
        activation = hp.Choice(name = 'activation_3_conv', values = ['relu', 'elu','tanh' ], ordered = False, default='relu'), #'selu'
        #activation = 'relu',
        )
    )
    model.add(
        tf.keras.layers.MaxPool2D(
            strides=hp.Int('MaxPol_3_stride', min_value=2, max_value=2, step=2),
        )
    )
    
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("dense_1_units", min_value=32, max_value=264, step=64),
            #activation = 'relu',
            activation = hp.Choice(name = 'activation_1_dense', values = ['relu','tanh' ], ordered = False, default='relu'),
        )
    )
    model.add(
        layers.Dense(
            units=hp.Int("dense_2_units", min_value=32, max_value=264, step=64),
            #activation = 'relu',
            activation = hp.Choice(name = 'activation_2_dense', values = ['relu','tanh' ], ordered = False, default='relu'),
        )
    )
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-4, 1e-3, 1e-2])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [49]:
SEED = 768
tuner = Hyperband(
    build_model_3c3p1f2d1d_basic,
    objective="val_accuracy",
    max_epochs=10,
    hyperband_iterations = 1,
    seed=SEED,
    overwrite=True,
    directory="my_dir",
    project_name="HyperbandTuner_basic",
)

tuner.search_space_summary()

tuner.search(X_train, y_train, validation_split=0.3, batch_size=20)

Trial 30 Complete [00h 00m 49s]
val_accuracy: 0.9759036302566528

Best val_accuracy So Far: 0.9759036302566528
Total elapsed time: 00h 07m 58s


In [50]:
tuner.results_summary(num_trials=100)
!rm -rf my_dir/ 

Results summary
Results in my_dir/HyperbandTuner_basic
Showing 100 best trials
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 9
activation_1_conv: elu
MaxPol_1_stride: 4
conv_2_filter: 48
conv_2_kernel: 3
activation_2_conv: tanh
MaxPol_2_stride: 2
conv_3_filter: 80
conv_3_kernel: 3
activation_3_conv: relu
MaxPol_3_stride: 2
dense_1_units: 160
activation_1_dense: relu
dense_2_units: 160
activation_2_dense: tanh
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9759036302566528
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 7
activation_1_conv: elu
MaxPol_1_stride: 2
conv_2_filter: 96
conv_2_kernel: 5
activation_2_conv: relu
MaxPol_2_stride: 4
conv_3_filter: 64
conv_3_kernel: 2
activation_3_conv: tanh
MaxPol_3_stride: 2
dense_1_units: 96
activation_1_dense: tanh
dense_2_units: 224
activation_2_dense: tanh
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score